# Trabajo Práctico 1: Análisis Exploratorio - Organización de Datos

**Alumnos y Padrón**
+ Craviotto Roitbarg, Mateo Exequiel - 106255 
+ Gómez, Joaquín - 103735

https://github.com/joaqogomez/Organizacion-de-datos-fiuba


## Introducción

En el presente trabajo se propone realizar un análisis de los datos recolectados por una agencia sobre las personas que pagan los impuestos.
El objetivo de la agencia es utilizar la información recolectada para dirigir campañas y poder dirigir a los fiuagentes recaudadores a inspeccionar.


## Objetivos
Los objetivos de este notebook es:
- Entender los datos.
- Ver como se relacionan entre si.
- Poder sacar alguna conclusión o descubrir un patrón.
- Indicar cuáles son los factores más importantes que determinan si una persona tiene altos o bajos ingresos. 
- Armar un baseline.

## Configuraciones Iniciales
Primero, importamos las bibliotecas que vamos a utilizar:

In [82]:
import numpy as np
import pandas as pd
import requests

## Carga del set de datos
Cargamos el dataset:

In [85]:
with requests.get(
    "https://docs.google.com/spreadsheets/d/1-DWTP8uwVS-dZY402-dm0F9ICw_6PNqDGLmH0u8Eqa0/export?format=csv"
) as r, open("Dataset/impuestos.csv", "wb") as f:
    for chunk in r.iter_content():
        f.write(chunk)
df = pd.read_csv("Dataset/impuestos.csv")

## Limpieza del set de datos

Despues de cargar el dataset, chequeamos que campos tenemos y si tienen valores NULL.

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 13 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   anios_estudiados                            32561 non-null  int64 
 1   barrio                                      31978 non-null  object
 2   categoria_de_trabajo                        30725 non-null  object
 3   edad                                        32561 non-null  int64 
 4   educacion_alcanzada                         32561 non-null  object
 5   estado_marital                              32561 non-null  object
 6   ganancia_perdida_declarada_bolsa_argentina  32561 non-null  int64 
 7   genero                                      32561 non-null  object
 8   horas_trabajo_registradas                   32561 non-null  int64 
 9   religion                                    32561 non-null  object
 10  rol_familiar_registrad

Se observa que tenemos valores null en las columnas barrio, categoria_de_trabajo y trabajo.

In [61]:
#Verificamos que nuestra variable target, en este caso tiene_alto_valor_adquisitivo no tenga valores invalidos
df.tiene_alto_valor_adquisitivo.value_counts()
#Se puede ver que solo tiene dos valores disponibles que son 0 y 1 como necesitamos

0    24720
1     7841
Name: tiene_alto_valor_adquisitivo, dtype: int64

In [62]:
df.trabajo.value_counts()

profesional_especializado    4140
reparador                    4099
directivo_gerente            4066
entretenimiento              3770
ventas                       3650
otros                        3295
inspector                    2002
transporte                   1597
limpiador                    1370
sector_primario               994
soporte_tecnico               928
seguridad                     649
servicio_domestico            149
ejercito                        9
Name: trabajo, dtype: int64

In [63]:
df.categoria_de_trabajo.value_counts()

relacion_de_dependencia    22696
monotibutista               2541
empleado_municipal          2093
empleado_provincial         1298
responsable_inscripto       1116
empleadao_estatal            960
trabajo_voluntariado          14
sin_trabajo                    7
Name: categoria_de_trabajo, dtype: int64

In [64]:
df.barrio.value_counts()

Palermo                29170
Belgrano                 643
San Isidro               198
Villa Urquiza            137
Recoleta                 121
La Boca                  114
Agronomia                106
Almagro                  100
Balvanera                 95
Puerto Madero             90
Caballito                 81
Boedo                     80
Barracas                  75
Chacarita                 73
Coghland                  70
Floresta                  67
Constitucion              64
Colegiales                62
Flores                    60
Liniers                   59
Monte Castro              51
Mataderos                 44
Nueva Pompeya             43
Monserrat                 37
nuñez                     34
Parque Chacabuco          31
Villa Luro                29
Parque Avellaneda         29
Parque Chas               28
La Paternal               24
Retiro                    20
Villa Devoto              19
Villa Soldati             19
San Telmo                 18
Villa Real    

In [70]:
df.educacion_alcanzada.value_counts()


4_anio                10501
5_anio                 7291
universidad_3_anio     5355
universidad_4_anio     1723
universidad_1_anio     1382
2_anio                 1175
universidad_2_anio     1067
1_anio                  933
7-8_grado               646
universiada_5_anio      576
9_grado                 514
3_anio                  433
universiada_6_anio      413
5-6_grado               333
1-4_grado               168
preescolar               51
Name: educacion_alcanzada, dtype: int64

In [73]:
df[df['barrio'].isna() & df['trabajo'].isna() &df['tiene_alto_valor_adquisitivo'] == 1]

,anios_estudiados,barrio,categoria_de_trabajo,edad,educacion_alcanzada,estado_marital,ganancia_perdida_declarada_bolsa_argentina,genero,horas_trabajo_registradas,religion,rol_familiar_registrado,tiene_alto_valor_adquisitivo,trabajo
1676,13,NaN,NaN,64,4_anio,matrimonio_civil,0,hombre,40,cristianismo,casado,1,NaN
18615,17,NaN,NaN,27,universidad_3_anio,matrimonio_civil,0,mujer,35,judaismo,casada,1,NaN
25359,17,NaN,NaN,38,universidad_3_anio,matrimonio_civil,7688,mujer,50,otro,casada,1,NaN
26363,14,NaN,NaN,48,5_anio,matrimonio_civil,0,hombre,40,cristianismo,casado,1,NaN


In [81]:
df[(df['ganancia_perdida_declarada_bolsa_argentina'] != 0) & df['categoria_de_trabajo'].isna()]


,anios_estudiados,barrio,categoria_de_trabajo,edad,educacion_alcanzada,estado_marital,ganancia_perdida_declarada_bolsa_argentina,genero,horas_trabajo_registradas,religion,rol_familiar_registrado,tiene_alto_valor_adquisitivo,trabajo
106,10,Palermo,NaN,17,1_anio,sin_matrimonio,34095,mujer,32,cristianismo,con_hijos,0,NaN
297,18,NaN,NaN,39,universidad_4_anio,matrimonio_civil,3464,mujer,40,budismo,casada,0,NaN
408,14,Palermo,NaN,28,5_anio,matrimonio_civil,-1887,mujer,40,cristianismo,casada,1,NaN
484,12,Palermo,NaN,52,3_anio,sin_matrimonio,594,hombre,40,judaismo,otro,0,NaN
656,9,Palermo,NaN,70,9_grado,viudo_a,1111,mujer,15,cristianismo,soltero_a,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30628,17,Palermo,NaN,65,universidad_3_anio,matrimonio_civil,5556,hombre,48,cristianismo,casado,1,NaN
30781,14,Palermo,NaN,66,5_anio,viudo_a,-4356,mujer,40,judaismo,soltero_a,0,NaN
31062,13,Palermo,NaN,59,4_anio,matrimonio_civil,2407,hombre,60,cristianismo,casado,0,NaN
31605,12,Palermo,NaN,20,3_anio,sin_matrimonio,-1602,hombre,40,otro,sin_familia,0,NaN
